# Automatically produce issues...

...for the TyIn in the Symogih.org ontology, according to the RDFs ontology schema.

Some cells originate from [graph_to_ontome.ipynb](https://github.com/Semantic-Data-for-Humanities/OntoME_importer/blob/main/graph_to_ontome.ipynb), despite some adaptation:
* cell n°4 (open and parse RDF file),
* cell n°6 (find and record TyIn informations).

## Declarations

In [10]:
from tqdm.notebook import tqdm #tqdm gives a progression bar to for loops.
from datetime import datetime
from itertools import chain
import json
import numpy as np
import rdflib
import requests
import time

file = "sources/xml_symogih.xml"

## Parse RDF schema

In [2]:
### Open and import file.
try:
    g = rdflib.Graph()
    g.parse(file, format="xml")
    
    # Prepare namespace declarations.
    ns_decl_list = []
    
    # Prepare checks.
    typed_decls = {}
    typelist = []
    for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), None)):

        # These elements will have different values when they're dealt with.
        typed_decls[s] = "no"
        typelist.append(o)
    print("These rdf:type objects were detected :\n")
    print(np.unique(typelist))
    
except Exception as e:
    print('Error: ' + str(e))

These rdf:type objects were detected :

['http://www.w3.org/2002/07/owl#Class'
 'http://www.w3.org/2002/07/owl#ObjectProperty'
 'http://www.w3.org/2002/07/owl#Ontology']


## Get simple TyIn information

In [3]:
### Test the recovery of 'classes',
# as well as the predicates of which they are subjects.

tyin = {}
cpredispo = [
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#type",
    "http://www.w3.org/2000/01/rdf-schema#comment",
    "http://www.w3.org/2000/01/rdf-schema#isDefinedBy",
    "http://www.w3.org/2000/01/rdf-schema#label",
    "http://www.w3.org/2000/01/rdf-schema#subClassOf",
    "http://www.w3.org/2002/07/owl#equivalentClass",
    "http://www.w3.org/2002/07/owl#unionOf"
]
cpredicates = []

# Get the predicates.
for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))):
    if 'TyIn' in str(s):
        tyin[s] = {}
        for so, po, oo in g.triples((s, None, None)):
            cpredicates.append(po)
            if po in tyin[s].keys():
                tyin[s][po].append(so)
            else:
                tyin[s][po] = [oo]

# Direct output.
print(f"There are {len(tyin.keys())} 'TyIn'.\n")

print("These are the associated properties :\n\t")
    
for p in np.unique(cpredicates):
    print("\t", p)

There are 133 'TyIn'.

These are the associated properties :
	
	 http://www.w3.org/1999/02/22-rdf-syntax-ns#type
	 http://www.w3.org/2000/01/rdf-schema#comment
	 http://www.w3.org/2000/01/rdf-schema#label
	 http://www.w3.org/2000/01/rdf-schema#subClassOf
	 http://www.w3.org/2004/02/skos/core#notation


## Add relationship information

In [4]:
for inftype in tyin.keys():
    ident = inftype.split('/')[-1]
    altident = ident + "_"
    
    local = {}
    
    for s, p, o in g.triples((None, None, None)):
        subjident = str(s).split('/')[-1]
        if subjident != ident :
            if subjident.endswith(ident) == True or subjident.startswith(altident):
                if s in local.keys():
                    local[s][p] = o
                else:
                    local[s] = {p:o}
            
    tyin[inftype]['to_table'] = local

## Initiate progress logs

In [7]:
dont_post = [
    'TyIn6',
    'TyIn3',
    'TyIn7',
    'TyIn11',
    'TyIn1',
    'TyIn2',
    'TyIn12',
    'TyIn130',
    'TyIn13',
    'TyIn55'
]
posted = []
failed = []

## Create and post issue contents

In [8]:
issues = {}

# Token removed for discretion.
token = ""

# After script execution,
# repository will be transferred to original owner:
# Semantic-Data-for-Humanitites.
owner = "MPica"

repo = "Alignement_symogih.org"

url = f"https://api.github.com/repos/{owner}/{repo}/issues"
headers = {
    "Authorization" : f"token {token}",
    "accept":"application/vnd.github+json"
}


# Start looping on TyIns and get their ID and title.

for inftype in tqdm(tyin.keys()):
    local = tyin[inftype]
    itLabel = local[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label')][0].value
    itid = str(inftype).split('/')[-1]
    
    title = f"{itid} {itLabel}"
    print(f"{title} en cours")
    
    # Initiate the body of the issue.
    sortie = "### Résumé\n\n**Scope note**\n\n"
    
    # Defining string values for descriptive items.
    scopeNote = local[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment')][0].value
    lien = str(inftype)
    
    sortie += f"{scopeNote}\n\n**Lien**\n\n[{str(inftype)}]({str(inftype)})\n\n"
    sortie += "### Alignement\n\n**Alignement proposé** :\n\n#### Alignement des rôles\n\n| Rôle (tel qu'apparaissant dans OntoME) | Range | Alignement | Notes |\n| ----- | ----- | ----- | ----- |"

    # Making the role table.
    
    for role in local['to_table'].keys():
        rolepreds = local['to_table'][role]
        roleID = str(role).split('/')[-1]
        
        notation = rolepreds[rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#notation')].value
        roleLabel = rolepreds[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label')].value
        roleRange = str(rolepreds[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range')])
        shortrange = roleRange.split('/')[-1]
        sortie += f"\n| [sym:{roleID} {roleLabel}]({str(role)}) | [sym:{shortrange}]({roleRange}) |   |   |"
        
    sortie += "\n\n### Commentaires"
    
    # Output a json archive of the data transmitted.
    with open(f"data/{itid}.json", "w") as openfile:
        json.dump({
            "title" : title,
            "body" : sortie
        }, openfile)
        
    # Output a markdown file for checking and archiving purposes.
    with open(f"data/{itid}.md", "w") as openfile:
        openfile.write(sortie)
    
    # If the current TyIn has no issue, post it.
    
    if itid not in dont_post and itid not in posted:
        payload = json.dumps({"title" : title, "body" : sortie, "labels":["TyIn", "En cours"]})
        response = requests.request("POST", url, data=payload, headers=headers)

        if response.status_code == 202 or response.status_code == 201:
            print('Successfully created Issue {0:s}'.format(title))
        else:
            print('Could not create Issue {0:s}'.format(title))
            print('Response says : ', response.content)
            failed.append(itid)
            
        print(response)
        posted.append(itid)
        
        print(f"👍 {title} a été posté !")
        time.sleep(30) # seconds
    else:
        print(f"{title} existe déjà.")

  0%|          | 0/133 [00:00<?, ?it/s]

TyIn1 Lettre en cours
TyIn1 Lettre existe déjà.
TyIn100 Opinion exprimée à l'égard d'un objet en cours
TyIn100 Opinion exprimée à l'égard d'un objet existe déjà.
TyIn102 Intervention manuscrite en cours
Successfully created Issue TyIn102 Intervention manuscrite
<Response [201]>
👍 TyIn102 Intervention manuscrite a été posté !
TyIn103 Exemplaire en cours
Successfully created Issue TyIn103 Exemplaire
<Response [201]>
👍 TyIn103 Exemplaire a été posté !
TyIn104 Présence en cours
Successfully created Issue TyIn104 Présence
<Response [201]>
👍 TyIn104 Présence a été posté !
TyIn105 Absence en cours
Successfully created Issue TyIn105 Absence
<Response [201]>
👍 TyIn105 Absence a été posté !
TyIn107 Représentation visuelle multiple en cours
Successfully created Issue TyIn107 Représentation visuelle multiple
<Response [201]>
👍 TyIn107 Représentation visuelle multiple a été posté !
TyIn108 Collection en cours
Successfully created Issue TyIn108 Collection
<Response [201]>
👍 TyIn108 Collection a été 

Successfully created Issue TyIn165 Relation entre acteurs collectifs
<Response [201]>
👍 TyIn165 Relation entre acteurs collectifs a été posté !
TyIn166 Activité d'enseignement d'un acteur collectif en cours
Successfully created Issue TyIn166 Activité d'enseignement d'un acteur collectif
<Response [201]>
👍 TyIn166 Activité d'enseignement d'un acteur collectif a été posté !
TyIn168 Période d'activité d'une relation en cours
Successfully created Issue TyIn168 Période d'activité d'une relation
<Response [201]>
👍 TyIn168 Période d'activité d'une relation a été posté !
TyIn169 Exercice de pouvoir sur un territoire en cours
Successfully created Issue TyIn169 Exercice de pouvoir sur un territoire
<Response [201]>
👍 TyIn169 Exercice de pouvoir sur un territoire a été posté !
TyIn170 Destruction en cours
Successfully created Issue TyIn170 Destruction
<Response [201]>
👍 TyIn170 Destruction a été posté !
TyIn2 Qualification doctrinale en cours
TyIn2 Qualification doctrinale existe déjà.
TyIn25 Éch

TyIn85 Moyenne (obtention d'une) en cours
Successfully created Issue TyIn85 Moyenne (obtention d'une)
<Response [201]>
👍 TyIn85 Moyenne (obtention d'une) a été posté !
TyIn86 Nationalité politique en cours
Successfully created Issue TyIn86 Nationalité politique
<Response [201]>
👍 TyIn86 Nationalité politique a été posté !
TyIn87 Réception mondaine en cours
Successfully created Issue TyIn87 Réception mondaine
<Response [201]>
👍 TyIn87 Réception mondaine a été posté !
TyIn88 Service militaire actif en cours
Successfully created Issue TyIn88 Service militaire actif
<Response [201]>
👍 TyIn88 Service militaire actif a été posté !
TyIn89 Fermage en cours
Successfully created Issue TyIn89 Fermage
<Response [201]>
👍 TyIn89 Fermage a été posté !
TyIn90 Transfert d'un(e) religieux(se) en cours
Successfully created Issue TyIn90 Transfert d'un(e) religieux(se)
<Response [201]>
👍 TyIn90 Transfert d'un(e) religieux(se) a été posté !
TyIn91 Assemblée en cours
Successfully created Issue TyIn91 Assembl

## Check on job execution

In [9]:
print("Les issues suivantes ont été postées :")
for issue in posted:
    print(f"\t\t{issue}")
    
print("\n\nLes issues suivantes ont échoué :")
for issue in failed:
    print(f"\n\n{issue}")

Les issues suivantes ont été postées :
		TyIn100
		TyIn102
		TyIn103
		TyIn104
		TyIn105
		TyIn107
		TyIn108
		TyIn110
		TyIn111
		TyIn112
		TyIn114
		TyIn115
		TyIn116
		TyIn117
		TyIn118
		TyIn121
		TyIn122
		TyIn123
		TyIn124
		TyIn125
		TyIn126
		TyIn127
		TyIn128
		TyIn129
		TyIn131
		TyIn132
		TyIn133
		TyIn134
		TyIn136
		TyIn137
		TyIn138
		TyIn139
		TyIn14
		TyIn140
		TyIn141
		TyIn142
		TyIn143
		TyIn144
		TyIn145
		TyIn146
		TyIn147
		TyIn148
		TyIn149
		TyIn15
		TyIn150
		TyIn151
		TyIn152
		TyIn153
		TyIn154
		TyIn155
		TyIn156
		TyIn157
		TyIn158
		TyIn160
		TyIn165
		TyIn166
		TyIn168
		TyIn169
		TyIn170
		TyIn25
		TyIn26
		TyIn27
		TyIn28
		TyIn30
		TyIn31
		TyIn32
		TyIn33
		TyIn34
		TyIn36
		TyIn37
		TyIn41
		TyIn42
		TyIn44
		TyIn49
		TyIn5
		TyIn51
		TyIn52
		TyIn53
		TyIn54
		TyIn56
		TyIn57
		TyIn58
		TyIn59
		TyIn60
		TyIn61
		TyIn62
		TyIn63
		TyIn64
		TyIn66
		TyIn67
		TyIn68
		TyIn69
		TyIn70
		TyIn71
		TyIn72
		TyIn73
		TyIn74
		TyIn75
		TyIn76
		TyIn77
		TyI

(Empty = no issue failed.)

## Documentation used:

-----

https://medium.com/analytics-vidhya/getting-started-with-github-api-dc7057e2834d

https://docs.github.com/fr/rest/issues/issues?apiVersion=2022-11-28#create-an-issue

------

https://medium.datadriveninvestor.com/github-issue-via-python-171c7fee0629

https://docs.github.com/fr/rest/guides/best-practices-for-integrators?apiVersion=2022-11-28#dealing-with-secondary-rate-limits

https://docs.github.com/fr/rest/overview/resources-in-the-rest-api?apiVersion=2022-11-28#rate-limiting

https://docs.github.com/fr/actions/security-guides/automatic-token-authentication